# Oscillation
This notebook contains experiments comparing methods for detecting oscillations. Using historical data, we simulate the following real-time detection scenario: streams of voltage/current measurements are retrieved, and we need to decide in (near) real-time whether an oscillation is beginning to occur. 

In [ ]:
import os, sys
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import seaborn as sns
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
from modules.preprocessing import *
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from tqdm.notebook import tqdm
import seaborn
import pickle
from copy import deepcopy
from timeit import default_timer as timer
from sklearn.ensemble import RandomForestRegressor as RFRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.pipeline import Pipeline
import lightgbm as lgb
plt.style.use('ggplot')
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np
import glob
import datetime
import math as mt
import scipy.linalg as sl
import scipy.optimize as opt
import time
from glob import glob
from os_utils import * 

# Test the performance of ocsillation method

## Import Dask

In [ ]:
from dask.distributed import Client
from dask.distributed import wait
import dask
from dask import delayed
dask.config.set(scheduler='synchronous')

## Start Dask

In [ ]:
num_workers = 30
client = Client(n_workers=num_workers,threads_per_worker=1)
client

## Create streams in memory for the experimental evaluation

In [ ]:
#read once, create many copies

f = '/data/data2/oscillations_data/synthetic/large_os_data.csv'
#read 100000
df = pd.read_csv(f, nrows = 100000)
df = df.set_index('timestamp')
df.index = pd.to_datetime(df.index)
### this window size corresponds to 10 s

temp_stream = df.values

n_streams = 60000

test_data = np.tile([temp_stream], (n_streams, 1, 1))
for j in range(len(test_data)):
    for i in range(test_data.shape[2]):
        noise =  np.random.normal(0, 0.1, test_data.shape[1])
        test_data[j, :, i] = test_data[j, :, i] + noise
             


## Sequential execution

In [ ]:
def fit_sin_simple2(y, step):
    """
    Estimate a signal using an FFT-based method.

    Parameters:
        df (array-like): The input signal to estimate.
        

    Returns:
        A tuple (data_fit, y_norm) containing the estimated (normalized) signal and the normalized signal.
    """
    #index = (df.index - min(df.index)).total_seconds()
    index = np.array(range(len(y)))/step
    var = np.var(y)
    # Normalized data
    y_norm = (y - np.mean(y))/var
    acorr = sm.tsa.acf(y_norm, nlags = len(y_norm)-1)
    peaks = signal.find_peaks(acorr[:len(y_norm)])
    try:
        peak_idx = peaks[0][0]
        #est_freq = 1/(df.index[peak_idx]-df.index[0]).total_seconds()
        est_freq = 1/((peak_idx+1)/step)
    except:
        est_freq = 1
    
    #find amplitude
    #optimize_func = lambda x: (x[0]*np.sin(2*np.pi*index) - y_norm)
    #find phase
    yf = np.fft.fft(y_norm)
    T = 1/30
    freq = np.fft.fftfreq(y.size, d=T)
    ind, = np.where(np.isclose(freq, est_freq, atol=1/(T*len(y))))
    est_phase = np.angle(yf[ind[0]])
    est_amp = np.sqrt(2)*np.std(y_norm)
    #index =
    data_fit = est_amp*np.sin(2*np.pi*est_freq*index+est_phase) 
    return data_fit, y_norm


In [ ]:
%%time
#sequential

running_time = []
total_running_time = 0.0

window_size = 1000
step = 30
M = len(test_data[0])
j = 0
while j < M:  
    for i, arr in enumerate(test_data):
        y = arr[j:min(j+window_size, M)]
        
        # Approximate the signal using a simple fft-based sinusoidal regression
        start = time.time()
        approx_signal4, y_norm = fit_sin_simple2(y, step)
        end = time.time()
        
        running_time_temp = end - start
        running_time.append(running_time_temp)
        total_running_time = total_running_time + running_time_temp
    j = j + window_size
print ("total_running_time = ", total_running_time )

## Parallel Batch code 

In [ ]:
def fit_sin_simple2(y, step):
    """
    Estimate a signal using an FFT-based method.

    Parameters:
        df (array-like): The input signal to estimate.
        

    Returns:
        A tuple (data_fit, y_norm) containing the estimated (normalized) signal and the normalized signal.
    """
    #index = (df.index - min(df.index)).total_seconds()
    index = np.array(range(len(y)))/step
    var = np.var(y)
    # Normalized data
    y_norm = (y - np.mean(y))/var
    acorr = sm.tsa.acf(y_norm, nlags = len(y_norm)-1)
    peaks = signal.find_peaks(acorr[:len(y_norm)])
    try:
        peak_idx = peaks[0][0]
        #est_freq = 1/(df.index[peak_idx]-df.index[0]).total_seconds()
        est_freq = 1/((peak_idx+1)/step)
    except:
        est_freq = 1
    
    #find amplitude
    #optimize_func = lambda x: (x[0]*np.sin(2*np.pi*index) - y_norm)
    #find phase
    yf = np.fft.fft(y_norm)
    T = 1/30
    freq = np.fft.fftfreq(y.size, d=T)
    ind, = np.where(np.isclose(freq, est_freq, atol=1/(T*len(y))))
    est_phase = np.angle(yf[ind[0]])
    est_amp = np.sqrt(2)*np.std(y_norm)
    #index =
    data_fit = est_amp*np.sin(2*np.pi*est_freq*index+est_phase) 
    return data_fit, y_norm

In [ ]:
def oscillations_predict(batch_data, step):
    results = []
    for batch in batch_data:
        approx_signal4, y_norm = fit_sin_simple2(batch, step)
        results.append(pprox_signal4, y_norm)
    return results

In [ ]:
def parallel_batch_processing_delayed(batch_data, step):
    running_time = 0.0
    futures = []
    
    delayed_process = dask.delayed(oscillations_predict)
    
    delayed_results = [delayed_process(arr,step) for arr in batch_data]
    
    start = time.time()
    results = dask.compute(*delayed_results)  
    end = time.time()
    
    del futures
    del results
    
    running_time = end - start
    
    return running_time

In [ ]:
%%time
#parallel

batch_data_size = len(test_data)//num_workers
batch_data = []
batch_data_all = []
counter = 0;
num_worker = 0

running_time = []
total_running_time = 0.0

window_size = 100
step = 30
M = len(test_data[0])
j = 0
while j < M:    
    for i, arr in enumerate(test_data):

        y = arr[j:min(j+window_size, M)]
       
        if (counter < batch_data_size):
            batch_data.append(y)
            counter = counter + 1
        elif num_worker == num_workers - 1:
            batch_data.append(y)
        else:
            counter = 0
            batch_data_all.append(batch_data)
            
            batch_data = []
            batch_data.append(y)
            counter = counter + 1
            num_worker = num_worker + 1
     
    batch_data_all.append(batch_data)
    batch_data = []
    
    num_worker = 0
    counter = 0
    
    running_time_temp = parallel_batch_processing_delayed(batch_data_all, step)
    running_time.append(running_time_temp)
    total_running_time = total_running_time + running_time_temp
    batch_data_all = []
    
    j = j + window_size
print ("total_running_time = ", total_running_time )

## Experiments

In [ ]:
n_streams = 60000, window_size = 100:
    
    sequential:
        total_running_time =  9792.758454322815
        CPU times: user 2h 44min 49s, sys: 10.1 s, total: 2h 44min 59s
        Wall time: 2h 45min 3s
            
    threads = 5:
        total_running_time =  2535.8568437099457
        CPU times: user 11min 23s, sys: 2min 5s, total: 13min 29s
        Wall time: 45min 59s 
            
    threads = 8:
        total_running_time =  1771.6430671215057
        CPU times: user 10min 12s, sys: 2min 15s, total: 12min 27s
        sWall time: 32min 26s
            
    threads = 10:
        total_running_time =  1507.3006780147552
        CPU times: user 10min 45s, sys: 1min 35s, total: 12min 21s
        Wall time: 28min 13s
    
    threads = 15:
        total_running_time =  1141.111942768097
        CPU times: user 9min 14s, sys: 1min 46s, total: 11min 1s
        Wall time: 22min
        
    threads = 16:
        total_running_time =  1110.9579532146454
        CPU times: user 9min 29s, sys: 2min, total: 11min 30s
        Wall time: 20min 51s
        
    threads = 20:
        total_running_time =  984.4872004985809
        CPU times: user 9min 33s, sys: 2min 4s, total: 11min 38s
        Wall time: 18min 54s
            
    threads = 30:
        total_running_time =  857.7079653739929
        CPU times: user 9min 14s, sys: 2min 16s, total: 11min 31s
        Wall time: 16min 37s        
      
    
n_streams = 60000, threads = 30 :
        
    window_size = 40 :
        total_running_time =  1858.6870143413544
        CPU times: user 21min 47s, sys: 3min 11s, total: 24min 58s
        Wall time: 36min 52s
            
    window_size = 60 :
        total_running_time =  1290.3779816627502
        CPU times: user 15min 13s, sys: 2min 17s, total: 17min 30s
        Wall time: 25min 42s
            
    window_size = 80 :
        total_running_time =  1032.3376879692078
        CPU times: user 11min 54s, sys: 2min 11s, total: 14min 6s
        Wall time: 20min 22s
            
    window_size = 100 :
        total_running_time =  857.7079653739929
        CPU times: user 9min 14s, sys: 2min 16s, total: 11min 31s
        Wall time: 16min 37s  
            
    window_size = 120 :
        total_running_time =  731.5465259552002
        CPU times: user 8min 15s, sys: 1min 47s, total: 10min 3s
        Wall time: 14min 18s
        
    window_size = 140 :
        total_running_time =  667.1877629756927
        CPU times: user 7min 16s, sys: 1min 56s, total: 9min 13s
        Wall time: 12min 59s
 

threads = 30, window_size = 100: 
    
    n_streams = 20000:
        total_running_time =  322.40996408462524
        CPU times: user 3min 47s, sys: 34.5 s, total: 4min 21s
        Wall time: 6min 11s
        
    n_streams = 40000:
        total_running_time =  585.6359477043152
        CPU times: user 6min 46s, sys: 1min 14s, total: 8min 1s
        Wall time: 11min 18s
        
    n_streams = 60000:
        total_running_time =  857.7079653739929
        CPU times: user 9min 14s, sys: 2min 16s, total: 11min 31s
        Wall time: 16min 37s  
        
    n_streams =80000:
        total_running_time =  1145.0851163864136
        CPU times: user 12min 48s, sys: 2min 54s, total: 15min 42s
        Wall time: 22min 25s
        
    n_streams =100000:
        total_running_time =  1415.4163179397583
        CPU times: user 16min 6s, sys: 3min 33s, total: 19min 39s
        Wall time: 27min 52s

## Close Dask

In [ ]:
client.close()